# Search in complex environments

In [1]:
from search import *
from notebook import psource, heatmap, gaussian_kernel, show_map, final_path_colors, display_visual, plot_NQueens

# Needed to hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")

# Code environment for these examples

In [2]:
psource(Problem)

In [3]:
psource(Node)

## Hill-climbing

This is a greedy local search that just wants to know what direction will most steeply move towards a local optimum value.

In [4]:
psource(hill_climbing)

## Example: Traveling Salesman Problem

This is an NP-hard problem: what is the shortest route to visit all cities in a list? We'll use hill-climbing to approximate the solution. This problem uses our Romania map again.

<p align="center">
<img src="images/romania.jpg">
</p>

### All of the cities

In [5]:
distances = {}
all_cities = []

for city in romania_map.locations.keys():
    distances[city] = {}
    all_cities.append(city)
    
all_cities.sort()
print(all_cities)

['Arad', 'Bucharest', 'Craiova', 'Drobeta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 'Lugoj', 'Mehadia', 'Neamt', 'Oradea', 'Pitesti', 'Rimnicu', 'Sibiu', 'Timisoara', 'Urziceni', 'Vaslui', 'Zerind']


### Distances between cities

In [6]:
import numpy as np
for name_1, coordinates_1 in romania_map.locations.items():
        for name_2, coordinates_2 in romania_map.locations.items():
            distances[name_1][name_2] = np.linalg.norm(
                [coordinates_1[0] - coordinates_2[0], coordinates_1[1] - coordinates_2[1]])
            distances[name_2][name_1] = np.linalg.norm(
                [coordinates_1[0] - coordinates_2[0], coordinates_1[1] - coordinates_2[1]])

distances

{'Arad': {'Arad': 0.0,
  'Bucharest': 350.2941620980858,
  'Craiova': 260.4995201531089,
  'Drobeta': 206.70026608594387,
  'Eforie': 511.31399354995165,
  'Fagaras': 218.27734651126764,
  'Giurgiu': 360.4719129141687,
  'Hirsova': 465.20210661603846,
  'Iasi': 382.25645841502796,
  'Lugoj': 135.07405376311175,
  'Mehadia': 171.283390905248,
  'Neamt': 318.1980515339464,
  'Oradea': 88.54942122905152,
  'Pitesti': 260.4169733331528,
  'Rimnicu': 163.97560794215707,
  'Sibiu': 121.16517651536682,
  'Timisoara': 82.05485969764375,
  'Urziceni': 391.6490776192381,
  'Vaslui': 420.7469548315234,
  'Zerind': 42.5440947723653},
 'Bucharest': {'Arad': 350.2941620980858,
  'Bucharest': 0.0,
  'Craiova': 152.0855022676389,
  'Drobeta': 236.66220653074288,
  'Eforie': 165.5294535724685,
  'Fagaras': 154.62535367784935,
  'Giurgiu': 62.24146527838174,
  'Hirsova': 135.95955280891445,
  'Iasi': 193.31321734428818,
  'Lugoj': 240.68444071023785,
  'Mehadia': 232.31013753170566,
  'Neamt': 210.08569

In [7]:
class TSP_problem(Problem):

    """ subclass of Problem to define various functions 

    The state is a list of cities
    Its action is to choose two random indices, and to reverse the city list within those
    random indices. The cost function is the total distance between all cities following this path, 
    and looping back to the start. The value is the negative of the length, so maximizing
    the value minimizes the distance.

    You'll notice that we can't expand a search frontier here, so we have to change the hill-climbing
    algorithm a bit.
    """
    

    def two_opt(self, state):
        """ Neighbour generating function for Traveling Salesman Problem """
        neighbour_state = state[:]
        left = random.randint(0, len(neighbour_state) - 1)
        right = random.randint(0, len(neighbour_state) - 1)
        if left > right:
            left, right = right, left
        neighbour_state[left: right + 1] = reversed(neighbour_state[left: right + 1])
        return neighbour_state

    def actions(self, state):
        """ action that can be excuted in given state """
        return [self.two_opt]

    def result(self, state, action):
        """  result after applying the given action on the given state """
        return action(state)

    def path_cost(self, c = None, state1 = None, action = None, state2 = None):
        """ total distance for the Traveling Salesman to be covered if in state2  """
        cost = 0
        for i in range(len(state2) - 1):
            cost += distances[state2[i]][state2[i + 1]]
        # Complete the loop
        cost += distances[state2[0]][state2[-1]]
        return cost

    def value(self, state):
        """ value of path cost given negative for the given state """
        return -1 * self.path_cost(state2 = state)

### Adjusted hill-climbing logic to evaluate different permutations of our city list

In [8]:
def hill_climbing(problem, num_neighbors = 100):
    
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    
    def find_neighbors(state, number_of_neighbors=num_neighbors):
        """ finds neighbors using two_opt method """
        
        neighbors = []
        
        for i in range(number_of_neighbors):
            new_state = problem.two_opt(state)
            neighbors.append(Node(new_state))
            state = new_state
            
        return neighbors


    # as this is a stochastic algorithm, we will set a cap on the number of iterations
    iterations = 10000
    
    current = Node(problem.initial)
    while iterations:
        neighbors = find_neighbors(current.state)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
        # The book's code had the wrong inequality -- they forgot they stated the
        # problem as a maximization problem
        if problem.value(neighbor.state) >= problem.value(current.state):
            current.state = neighbor.state
        iterations -= 1
        
    return current.state

### Climbing the hill

The algorithm is stochastic, and you'll see that running this does not compare to the book's solution!

In [9]:
tsp = TSP_problem(all_cities)

In [10]:
hill_climbing(tsp)

['Craiova',
 'Pitesti',
 'Giurgiu',
 'Bucharest',
 'Urziceni',
 'Eforie',
 'Hirsova',
 'Vaslui',
 'Iasi',
 'Neamt',
 'Fagaras',
 'Rimnicu',
 'Sibiu',
 'Oradea',
 'Zerind',
 'Arad',
 'Timisoara',
 'Lugoj',
 'Mehadia',
 'Drobeta']

!["Book's hill"](images/hillclimb-tsp.png)



### Hill climbing isn't gradient descent

Gradient descent can move the the direction of maximum change, but hill climbing only changes one element in a vector at a time. Therefore, **ridges** can pose a problem because the method has to zig-zag to move along a ridge.

Another problem for hill climbing is **plateaus**. If there isn't a clear direction of change, the method can wander about.

And, as we already noted, the hill climbing algorithm will get stuck in the first optimum value that it finds, even if it's a local optimum. 

You can imagine that there are little corrections we could make to the algorithm that would address some of these problems, and the book enumerates a few.

### Peak-finding and hill-climbing

In [11]:
psource(PeakFindingProblem)

This is a 3-D grid that is accessed by choosing 0, 1, or 2 for which sub-array, and then 0, 1, 2, or 3 for which position. The maximum array value is 9, the minimum is 1.

In [12]:
initial = (0, 0)
grid = [[3, 7, 2, 8], [5, 2, 9, 1], [5, 3, 3, 1]]

In [13]:
grid[0][0]

3

In [14]:
grid[1][0]

5

In [15]:
grid[2][0]

5

#### Let's define hill-climbing for the peak problem

We'll show that we very quickly get stuck in a local min - and we can see that we should make our code smarter to not keep trying the same thing over and over

In [16]:
def hill_climbing(problem):
    
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    
    def find_neighbors(state):
        """ finds neighbors using two_opt method """
        
        neighbors = []
        for action in problem.actions(state):
            neighbors.append(Node(problem.result(state, action)))
        print("NEIGHBORS", [n.state for n in neighbors])
        return neighbors


    # as this is a stochastic algorithm, we will set a cap on the number of iterations
    iterations = 10
    
    current = Node(problem.initial)
    print("GRID", grid)
    print(current.state)
    while iterations:
        neighbors = find_neighbors(current.state)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
        # problem as a maximization problem
        if problem.value(neighbor.state) >= problem.value(current.state):
            current.state = neighbor.state
            print("IMPROVED", neighbor.state)
        iterations -= 1
        
    return current.state

In [17]:
problem = PeakFindingProblem(initial, grid, directions4)

In [18]:
hill_climbing(problem)

GRID [[3, 7, 2, 8], [5, 2, 9, 1], [5, 3, 3, 1]]
(0, 0)
NEIGHBORS [(0, 1), (1, 0)]
IMPROVED (0, 1)
NEIGHBORS [(0, 2), (1, 1), (0, 0)]
NEIGHBORS [(0, 2), (1, 1), (0, 0)]
NEIGHBORS [(0, 2), (1, 1), (0, 0)]
NEIGHBORS [(0, 2), (1, 1), (0, 0)]
NEIGHBORS [(0, 2), (1, 1), (0, 0)]
NEIGHBORS [(0, 2), (1, 1), (0, 0)]
NEIGHBORS [(0, 2), (1, 1), (0, 0)]
NEIGHBORS [(0, 2), (1, 1), (0, 0)]
NEIGHBORS [(0, 2), (1, 1), (0, 0)]


(0, 1)

## Simulated Annealing

Here's a nice [YouTube explanation and video](https://www.youtube.com/watch?v=iaq_Fpr4KZc):
"This video shows a run of simulated annealing optimization algorithm. The simulated annealing algorithm produces a sequence of points on the objective function surface. At the beginning, the points tend to jump around at relatively large distances, corresponding to high temperature. Later on, the point cools down, takes smaller jumps, and settles down at the minimum. The color of the point gives some indication of the temperature.

The data for this simulation was obtained from a program written by Roman Gassmann, the visualization and the conversion into a movie was done by Andreas Müller."

Note that their simulated annealing implementation only returns one node at a time.


In [19]:
psource(simulated_annealing)

In [20]:
psource(exp_schedule)

In [21]:
simulated_annealing(problem)

(1, 2)

In [22]:
for i in range(20):
    print(simulated_annealing(problem))

(0, 3)
(1, 2)
(1, 2)
(2, 1)
(2, 0)
(1, 2)
(0, 1)
(0, 3)
(2, 2)
(0, 1)
(1, 3)
(2, 2)
(1, 2)
(2, 1)
(1, 2)
(0, 3)
(2, 0)
(1, 1)
(1, 2)
(2, 2)


In [23]:
solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
solutions

{1, 2, 3, 5, 7, 8, 9}

### Beam searches hold onto more than one result at a time

Can you imagine how simulated annealing could settle down more quickly if we didn't lose information every time we moved?

# Evolutionary Algorithms

## Genetic Algorithms

Simplified example from my past research: suppose I want to know which days offer the greatest benefit for treating a disease and a limited budget for treatment? I could have a list of days and put a 1 if I'm treating and a 0 if I'm not. I could create a value function that looks something like
$$
J(\texttt{plan}) = \texttt{disease morbidity} + \texttt{disease cost}
$$
that I want to minimize. Note that the logic for $J$ may be based on a model or past data or whatever. We're not showing that here. Then I could just make up a bunch of answers to form a population of plans (I'll pretend there are only 7 days because I'm lazy).
$$
\texttt{Population} = \{ (1, 0, 0, 1, 0, 0, 1), (0, 0, 0, 1, 1, 1, 0), (0, 1, 0, 0, 0, 0, 0), ...\}
$$
You can imagine that as the number of days I can treat grows, I couldn't possibly list all combinations, so we just list some of these.

* **Selection** Select pairs from your original population to combine for a new population.
* **Crossover** is one way to combine; another is position-by-position. Position-by-position can be interested for non-binary vector values -- do you want to average, take on or the other, or a combination?  
![Uniform Crossover](images/uniform_crossover.png)
* **Mutation rate**
* **Elitism** just clones the great parents, and **Culling** makes sure we don't allow any undesirable parents reproduce. 
* **Immigration** can randomly add new vectors to the population.

## Particle Swarm Optimization

Is an interesting extension of this structure in which clusters of these little vector populations share information as well as global sharing, and each vector mutates in the direction of a better-performing neighbor.

[Here's a little explanation and graphic](https://pymoo.org/algorithms/soo/pso.html)



# Nondeterministic Actions

What if you're not sure what your action will accomplish?

Remember, the **transition model** is the model of what is going to happen as a result of each action. In most real-life applications, we aren't confident of the outcome of our actions, and the transition model typically is a list of possible outcomes weighted by the likelihood of those outcomes.

## Erratic Outcomes

For a search tree, the tree branches in an **and** node because the result of an action takes you down this path and that path.

<img src="images/erratic_vacuum.png" width="600">
<img src="images/andorsearch.png" width="600">



From the text resources:

The search is carried out by two functions `and_search` and `or_search` that recursively call each other, traversing nodes sequentially.
It is a recursive depth-first algorithm for searching an _AND-OR_ graph.
<br>
A very similar algorithm `fol_bc_ask` can be found in the `logic` module, which carries out inference on first-order logic knowledge bases using _AND-OR_ graph-derived data-structures.
<br>
_AND-OR_ trees can also be used to represent the search spaces for two-player games, where a vertex of the tree represents the problem of one of the players winning the game, starting from the initial state of the game.
<br>
Problems involving _MIN-MAX_ trees can be reformulated as _AND-OR_ trees by representing _MAX_ nodes as _OR_ nodes and _MIN_ nodes as _AND_ nodes.
`and_or_graph_search` can then be used to find the optimal solution.
Standard algorithms like `minimax` and `expectiminimax` (for belief states) can also be applied on it with a few modifications.


## Slippery Outcomes

<img src="images/slippery_vacuum.png" width="600">


## Exercise

Explain precisely how to modify the AND-OR-GRAPH-SEARCH algorithm to generate a
cyclic plan if no acyclic plan exists. You will need to deal with three issues: labeling the plan
steps so that a cyclic plan can point back to an earlier part of the plan, modifying OR-SEARCH
so that it continues to look for acyclic plans after finding a cyclic plan, and augmenting the
plan representation to indicate whether a plan is cyclic. Show how your algorithm works on
(a) the slippery vacuum world, and (b) the slippery, erratic vacuum world. You might wish to
use a computer implementation to check your results.



function AND-OR-GRAPH-SEARCH(problem) returns a conditional plan; or failure
OR-SEARCH(problem.INITIAL-STATE, problem, [ ])

function OR-SEARCH(state, problem, path) returns a conditional plan; or failure


> if problem.GOAL-TEST(state) then return the empty plan  
> if state has previously been solved then return RECALL-SUCCESS(state)  
> if state has previously failed for a subset of path then return failure  
> if state is on path then  
>> RECORD-FAILURE(state, path)  
>> return failure  
> for each action in problem.ACTIONS(state) do  
>> plan =  AND-SEARCH(RESULTS(state, action), problem, [state j path])  
>> if plan 6= failure then  
>>> RECORD-SUCCESS(state, [action j plan])
return [action j plan]
return failure
function AND-SEARCH(states, problem, path) returns a conditional plan; or failure
for each si in states do
plani OR-SEARCH(si, problem, path)
if plani = failure then return failure
return [if s1 then plan1 else if s2 then plan2 else : : : if sn􀀀1 then plann􀀀1 else plann]

# Partially Observable Environments

* **States** -- The "belief-state space" is every possible state configuration
* **Initial state** -- What does the agent know about the environment to start with? It may be that it knows all of the states, or perhaps this is a subset of all of the states.
* **Actions** -- Not all actions are accessible from each item in the state. So there's a choice here:
    * The actions here can be the *union* of the actions from each item in the current state, even though some of those actions won't be legal. 
    * If taking an illegal action is catastrophic, the list of actions can be the *intersection* of the possible actions from each item in the state.
* **Transition Model** -- Remember, the agent has a list of possible states that may be its initial state. So what happens when it choose one possible action and takes that action? It's the union of the result of that action when applied to each state in its belief state.
* **Goal Test** -- test each state resulting from the transition model. Do any of them achieve the goal?
    * If so then it's possible that the goal has been met.
    * But you're only certain if the goal has been met regardless of the state suggested by the transition model.
* **Action cost** -- the same action may have a different cost based on the actual state so the actual cost will be hard to determine.

![Example](images/fig14-1.png)


Imagine a slight change to the above picture. How would the graphic change if the vacuum could see its local environment?

# Offline and Online Searching

* **Offline** calculate the entire solution and press the start button
* **Online** Calculate an action, perform action, perceive, calculate, perform, perceive, ...

Of course, if the agent doesn't know what might happen as a result of its actions, it could make a bad mistake and end up in a dead-end.

## Online local search

Try hill-climbing with a memory -- Below each dot is a guess at the cost for reaching a goal. And at each move, it's discovering where the guesses could not have been right and it's correcting the goals. In this way, moving back and forth, it's finally able to escape. The book calls this **optimism under uncertainty**.

![LRTA*](images/lrtastart.png)
